In [1]:
import zero2one

from collections import Counter
import pandas as pd
from tqdm.notebook import tqdm

from nltk import collocations
from konlpy.tag import Twitter,Okt,Kkma
import ast
okt = Okt()
kkma = Kkma()

# Coupang Loading

In [2]:
td1_1 = ["Noun","Verb","Adverb","Adjective","Unknown","ETC","VerbPrefix"]
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\coupang"
test_ = zero2one.Data_Load("contents",path_,"xlsx").parsing_date("date")
test_pos = zero2one.Pos_Process(test_,"contents").pos_tagging(td1_1,"okt")

Split Sentences:   0%|          | 0/43473 [00:00<?, ?it/s]

Pos Tagging:   0%|          | 0/43473 [00:00<?, ?it/s]

In [3]:
df_ = zero2one.Model1(test_pos,"sen_word2","sen_tag2").df
df_.head()

Preprocessing:   0%|          | 0/43473 [00:00<?, ?it/s]

,date,id,Product Name,score,liked,sentence,sentag,review_num,sen_num
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]","[(가격, Noun), (저렴하다, Adjective), (간단하다, Adjecti...",1,1
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[남자, 사용, 하다, 아주, 굿, 이다]","[(남자, Noun), (사용, Noun), (하다, Verb), (아주, Noun...",1,2
2,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 대비, 예쁘다, 좋다]","[(가격, Noun), (대비, Noun), (예쁘다, Adjective), (좋다...",2,1
3,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,0,"[가격, 대비, 만족하다]","[(가격, Noun), (대비, Noun), (만족하다, Adjective)]",3,1
4,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[저, 아직도, 잘쓰다, 있다]","[(저, Noun), (아직도, Adverb), (잘쓰다, Verb), (있다, A...",4,1


# Sentiment_dic 

In [4]:
dic_ = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\Reference_DB\Sentiment_DB\군산대\SentiWord_Dict_1.xlsx")

In [5]:
def sentiment_prepro(dic_,traget_tag,remove_list):
    dic_["tagging"] = dic_["Senti_Word"].map(lambda x : okt.pos(x,stem=True))
    dic_["tagging_only"] = dic_["tagging"].map(lambda x : [ x2[0] for x2 in x if x2[1] in traget_tag ] )
    dic_["len"] = dic_["tagging_only"].map(lambda x : len(x))
    dic_ = dic_[dic_["len"] != 0]
    remove_index = dic_.tagging_only.apply(lambda x: set(x).issubset(remove_list))
    # remove_index 에 있는 True를 False로 False를 True로 바꾸는 작업
    dic_ = dic_[remove_index.apply(lambda x: True if x==False else False)]
    return dic_

In [6]:
target_tag = ["Noun","Verb","Adjective","Adverb","VerbPrefix"]
remove_list = ["하다","저"]
senti_dic = sentiment_prepro(dic_,target_tag,remove_list)
senti_dic.head()

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len
0,굿,긍정,1,NaN,"[(굿, Noun)]",[굿],1
1,가격이 싸다,긍정,1,NaN,"[(가격, Noun), (이, Josa), (싸다, Verb)]","[가격, 싸다]",2
2,가까스로,중립,0,NaN,"[(가까스로, Noun)]",[가까스로],1
3,가까이 사귀어,긍정,1,NaN,"[(가까이, Noun), (사귀다, Verb)]","[가까이, 사귀다]",2
4,가까이하다,긍정,1,NaN,"[(가까이, Noun), (하다, Verb)]","[가까이, 하다]",2


# "S" matching

In [15]:
def split_sentest(target_tag,test_1):
    split_li = []
    buffer = []
    for word in test_1:
        if word[1] in target_tag:
            buffer.extend([word[0]])
            split_li.append(buffer)
            buffer = []
        else:
            buffer.extend([word[0]])
    return split_li

In [21]:
## 군산대 사전이랑 matching 하는 작업
def matching_s(df_,dic_,target_tag,range_):
    date_li,id_li,product_li,re_n,sen_n,reason_n,reason_li,sent_li = [],[],[],[],[],[],[],[]
    tq = tqdm(range(len(df_.iloc[range_[0]:range_[1]])))
    r = 1
    for i in tq:
        reasons = split_sentest(target_tag,df_["sentag"][i])
        for reason in reasons:
            date_li.append(df_["date"][i])
            id_li.append(df_["id"][i])
            product_li.append(df_["Product Name"][i])
            re_n.append(df_["review_num"][i])
            sen_n.append(df_["sen_num"][i])
            reason_li.append(reason)
            try:
                qeury = dic_[dic_.tagging_only.apply(lambda x : set(x).issubset(reason) )]
                if len(qeury) !=0:
                    try:
                        n_nega = qeury["Orientation"].value_counts()["부정"]
                    except:
                        n_nega = 0
                    try:
                        n_posi = qeury["Orientation"].value_counts()["긍정"]
                    except:
                        n_posi = 0
                    if n_nega == n_posi:
                        senti_q = [reason[0],"중립",0]
                    else:
                        senti_q = [reason[0],qeury["Orientation"].mode().values[0],qeury["Intensity"].mode().values[0],len(qeury)]
                else:
                    senti_q = []
            except:
                senti_q = []
            sent_li.append(senti_q)
            reason_n.append(r)
            r +=1
        r = 1
    da = [date_li,id_li,product_li,re_n,sen_n,reason_n,reason_li,sent_li]
    df_2 = pd.DataFrame(da).T.rename(columns={0:"date",1:"id",2:"Product Name",
                                              3:"review_num",4:"sen_num",5:"reason_num",6:"reason",7:"s"})
    return df_2

In [23]:
# 저는 sample로 0~200개의 reason 후보들을 군산대 senti_dic이랑 matching 하였습니다.
target_tag = ["Noun","Verb","Adjective","Adverb","VerbPrefix"]
df_ma = matching_s(df_,senti_dic,target_tag,[0,200])
df_ma.loc[0:40]

  0%|          | 0/200 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,1,[가격],[]
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,2,[저렴하다],"[저렴하다, 긍정, 1, 1]"
2,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,3,[간단하다],[]
3,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,4,[손쉽다],[]
4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,5,[사용],[]
5,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,6,[하다],[]
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],[]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,8,[좋다],"[좋다, 긍정, 1, 12]"
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,1,[남자],[]
9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],[]


In [24]:
## 다음 sentiment 단어가 나오지 전까지 채우는 작업 
def filling_s(df_ma):
    df_reverse = df_ma.iloc[::-1]
    tq = range(len(df_reverse))
    s_li = []
    buffer_ = (0,0,None)
    for i in tq:
        rn_ = df_reverse.iloc[i,3]
        sn_ = df_reverse.iloc[i,4]
        s_ = df_reverse.iloc[i,7]
        if s_ != []:
            buffer_ = (rn_,sn_,s_)
            s_li.append(buffer_[2])
        else:
            if rn_ == buffer_[0]:
                if sn_ == buffer_[1]:
                    s_li.append(buffer_[2])
                else:
                    s_li.append([])
            else:
                buffer_ = (0,0,None)
                s_li.append([])
    df_reverse["s"] = s_li
    df_3 = df_reverse.iloc[::-1]
    return df_3

In [24]:
df_fill = filling_s(df_ma)
df_fill.loc[10:15]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s
10,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]"
11,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]"
12,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,5,[굿],"[굿, 긍정, 1, 1]"
13,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,6,[이다],[]
14,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2,1,1,[가격],"[예쁘다, 긍정, 1, 6]"
15,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2,1,2,[대비],"[예쁘다, 긍정, 1, 6]"


In [25]:
### 중립 처리를 위해서 일단 보류
### 좋은 가격이다 -> "좋다, 가격" 같은 경우를 대비해서, user와 마찬가지로, 비어 있는 곳을 채운다.

def fill_emp_s(df_fill):
    buffer = (0,0,None)
    for row,col in df_fill.iterrows():
        if col["s"] != []:
            buffer = (col["review_num"],col["sen_num"],col["s"])
        elif col["s"]==[]:
            if buffer[0] == col["review_num"]:
                if buffer[1] == col["sen_num"]:
                    df_fill.loc[row,"s"] = buffer[2]
    return df_fill

In [26]:
df_em = fill_emp_s(df_fill)
df_em.loc[18:23]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s
18,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,1,[가격],"[만족하다, 긍정, 1, 5]"
19,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,2,[대비],"[만족하다, 긍정, 1, 5]"
20,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,3,[만족하다],"[만족하다, 긍정, 1, 5]"
21,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,1,[저],[]
22,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,2,[아직도],[]
23,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,3,[잘쓰다],[]


In [27]:
# senti가 없는 부분은 중립으로 지정하기

def fill_neu_s(df_fill):
    for row,col in df_fill.iterrows():
        if col["s"] == []:
            df_fill.loc[row,"s"] = ["Null","중립",0]
    return df_fill

In [28]:
df_neu = fill_neu_s(df_fill)
df_neu.loc[18:23]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s
18,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,1,[가격],"[만족하다, 긍정, 1, 5]"
19,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,2,[대비],"[만족하다, 긍정, 1, 5]"
20,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,3,[만족하다],"[만족하다, 긍정, 1, 5]"
21,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,1,[저],"[Null, 중립, 0]"
22,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,2,[아직도],"[Null, 중립, 0]"
23,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,3,[잘쓰다],"[Null, 중립, 0]"


# "U" matching

In [29]:
u_dic = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\Reference_DB\Sentiment_DB\usage_dic.xlsx")
u_dic["com"] = u_dic["com"].map(lambda x : ast.literal_eval(x))
u_dic

,key,attr,sim,com
0,사용 대상,자녀,#아기 #딸 #아들 #아이,"[자녀, 아기, 딸, 아들, 아이]"
1,NaN,강아지,#반려동물 #개,"[강아지, 개, 반려동물, 반려 동물]"
2,NaN,여성,#여자 #엄마,"[여성, 여자, 엄마]"
3,NaN,남성,#남자 #아빠,"[남성, 남자, 아빠]"
4,사용 장소,화장대,#책상,"[화장대, 책상]"
5,NaN,화장실,#욕실,"[화장실, 욕실]"
6,NaN,수영장,#목욕탕,"[수영장, 목욕탕]"
7,NaN,여행,#여행용,"[여행, 여행용]"
8,NaN,차량,#이동중 #차 안,"[차량, 이동중, 차 안, 이동 중, 운전]"


In [30]:
# u 에 해당하는 것 matching 하기

def matching_u(df_em,u_dic):
    usage_li = []
    tq = tqdm(range(len(df_em)))
    for i in tq:
        reason = df_em.loc[i,"reason"]
        try:
            usage_q = u_dic[u_dic.com.apply(lambda x : set(reason).issubset(x) )]["attr"].mode().values.tolist()
        except:
            usage_q = []
        usage_li.append(usage_q)
    df_em["u"] = usage_li
    return df_em


In [31]:
df_u = matching_u(df_neu,u_dic)
df_u.loc[6:11]

  0%|          | 0/1403 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,8,[좋다],"[좋다, 긍정, 1, 12]",[]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,1,[남자],"[굿, 긍정, 1, 1]",[남성 ]
9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[]
10,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[]
11,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]",[]


In [32]:
#  user 앞부분 채우기

def filling_u(df_ma):
    df_reverse = df_ma.iloc[::-1]
    tq = range(len(df_reverse))
    s_li = []
    buffer_ = (0,0,None)
    for i in tq:
        rn_ = df_reverse.iloc[i,3]
        sn_ = df_reverse.iloc[i,4]
        s_ = df_reverse.iloc[i,8]
        if s_ != []:
            buffer_ = (rn_,sn_,s_)
            s_li.append(buffer_[2])
        else:
            if rn_ == buffer_[0]:
                if sn_ == buffer_[1]:
                    s_li.append(buffer_[2])
                else:
                    s_li.append([])
            else:
                buffer_ = (0,0,None)
                s_li.append([])
    df_reverse["u"] = s_li
    df_3 = df_reverse.iloc[::-1]
    return df_3

In [33]:
df_uf = filling_u(df_u)
df_uf.loc[6:11]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,8,[좋다],"[좋다, 긍정, 1, 12]",[]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,1,[남자],"[굿, 긍정, 1, 1]",[남성 ]
9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[]
10,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[]
11,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]",[]


In [34]:
## 같은 문장에 뒤에 다른 user가 없으면 뒷부분도 채워주기

def fill_emp_u(df_fill):
    buffer = (0,0,None)
    for row,col in df_fill.iterrows():
        if col["u"] != []:
            buffer = (col["review_num"],col["sen_num"],col["u"])
        elif col["u"]==[]:
            if buffer[0] == col["review_num"]:
                if buffer[1] == col["sen_num"]:
                    df_fill.loc[row,"u"] = buffer[2]
    return df_fill

In [35]:
df_em_u = fill_emp_u(df_uf)
df_em_u.loc[6:11]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,8,[좋다],"[좋다, 긍정, 1, 12]",[]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,1,[남자],"[굿, 긍정, 1, 1]",[남성 ]
9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[남성 ]
10,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[남성 ]
11,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]",[남성 ]


# "S", "U" 에 걸렸으면 reason에서 제거

In [36]:
# s 지우기

def delete_reason_s(df_1):
    drop_index = []
    buffer_ = ""
    for row,col in df_1.iterrows():
        
        try:
            if col["reason"][0] == col["s"][0]:
                if buffer_ == col["s"][0]:
                    drop_index.append(row)
                else:
                    df_1.loc[row,"reason"] = ""
                
                #print(col["reason"][0],row)
        except:
            pass
        buffer_ = col["s"][0]
    return df_1.drop(drop_index,axis=0).reset_index(drop=True)

In [37]:
df_drop_s = delete_reason_s(df_em_u)
df_drop_s.loc[0:8]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,1,[가격],"[저렴하다, 긍정, 1, 1]",[]
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,3,[간단하다],"[좋다, 긍정, 1, 12]",[]
2,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,4,[손쉽다],"[좋다, 긍정, 1, 12]",[]
3,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,5,[사용],"[좋다, 긍정, 1, 12]",[]
4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,6,[하다],"[좋다, 긍정, 1, 12]",[]
5,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[]
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,1,[남자],"[굿, 긍정, 1, 1]",[남성 ]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[남성 ]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[남성 ]


In [38]:
# u 지우기

def delete_reason_u(df_1,u_dic):
    drop_index = []
    for row,col in df_1.iterrows():
        try:
            ma_u = u_dic[u_dic.com.apply(lambda x: set(col["reason"]).issubset(x))]["attr"].values.tolist()
            if ma_u != []:
                if ma_u == col["u"]:
                    #print(ma_u)
                    drop_index.append(row)
                #print(col["reason"][0],row)
        except:
            pass
    return df_1.drop(drop_index,axis=0).reset_index(drop=True)

In [39]:
df_drop = delete_reason_u(df_drop_s,u_dic)
df_drop.loc[0:8]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,1,[가격],"[저렴하다, 긍정, 1, 1]",[]
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,3,[간단하다],"[좋다, 긍정, 1, 12]",[]
2,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,4,[손쉽다],"[좋다, 긍정, 1, 12]",[]
3,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,5,[사용],"[좋다, 긍정, 1, 12]",[]
4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,6,[하다],"[좋다, 긍정, 1, 12]",[]
5,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[]
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[남성 ]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[남성 ]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]",[남성 ]


# "a" matching 하기

### Zero2one_seed

In [40]:
zero_df = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\dic_test\드라이기_20201207.xlsx",sheet_name="data_")
zero_df.head()

,Unnamed: 0,Product Attribute 측면 속성,유사 키워드(attr_sim),토큰화 방식으로 각 속성을 나타내는 키워드 조합,count,긍정,부정
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별로..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋아..."


In [41]:
def remove_list(ta_list,re_list):
    for re in re_list:
        try:
            ta_list.remove(re)
        except:
            pass
    return ta_list

In [42]:
def Zero_dic(zero_df):
    zero_dic = {}
    for i in range(len(zero_df)):
        keys_ = zero_df.iloc[i,0].replace("/"," ")
        keys_li = keys_.split(" ")
        keys_li = remove_list(keys_li,["","및"])


        attrs_ = zero_df.iloc[i,1]
        attrs_li = attrs_.split(" ")
        attrs_li = remove_list(attrs_li,["","및","사용"])

        all_li = keys_li + attrs_li
        try:
            zero_dic[keys_] = list(set(zero_dic[keys_] + all_li))
        except:
            zero_dic[keys_] = list(set(all_li))
    
    all_ma = []
    for key,value in zero_dic.items():
        all_ma.extend(value)    
  
    return zero_dic,all_ma

In [43]:
zero_dic,all_ma = Zero_dic(zero_df)
zero_dic

{'바람': ['바람', '세기', '조절', '풍량', '회전'],
 '온도': ['단계', '온도', '냉풍', '온풍', '조절', '열풍'],
 '조작감': ['버튼식', '온도', '슬라이드식', '조절', '조작감', 'COOL버튼'],
 '디지털 센서': ['모발',
  '센서',
  '터치',
  '보호',
  '기술',
  '온도',
  '수분',
  '모발속',
  '조절',
  '디지털',
  '센싱',
  '모드'],
 '발생 성분': ['원적외선', '플리즈마', '발생', '음이온', '케어', '성분', '이온'],
 '안전 기능': ['전자파',
  '물질',
  '방지',
  '이중안전',
  '제어',
  '시스템',
  '미열',
  '자체',
  '기능',
  '열',
  '차단',
  '유해',
  '장치',
  '기기',
  '안전'],
 '위생': ['케이스', '분리', '위생', '흡인구', '탈착필터'],
 '스타일링 부속품': ['부착형',
  '스무딩',
  '디퓨저',
  '스타일링',
  '부속품',
  '집중노즐',
  '콘센트레이터',
  '자석',
  '노즐'],
 '손잡이 형태': ['손잡이', '고정식', '형태', '접이식'],
 '코드': ['코드', '연질', '길이', '코드릴', '무선'],
 '무선 기능': ['가능', '배터리잔량표시', '충전', '스탠드', '기능', '시간', '무선'],
 '거치': ['벽걸이용', '거치대', '거치', '고리'],
 '부가 기능': ['프리볼트', '방지', '부가', '고데기', '미끄럼', '기능', '마사지', '두피'],
 '외관 디자인': ['색상', '외관', '재질', '무게', '제품', '디자인', '그립감', '크기'],
 '소비전력': ['소비전력', '에너지', '효율'],
 '내구성': ['표면', '제품', '잔기스', '내구성'],
 '소음진동': ['진동', '소음', '냄새', '소음진동'],
 '패키지': ['포

In [44]:
all_ma

['바람',
 '세기',
 '조절',
 '풍량',
 '회전',
 '단계',
 '온도',
 '냉풍',
 '온풍',
 '조절',
 '열풍',
 '버튼식',
 '온도',
 '슬라이드식',
 '조절',
 '조작감',
 'COOL버튼',
 '모발',
 '센서',
 '터치',
 '보호',
 '기술',
 '온도',
 '수분',
 '모발속',
 '조절',
 '디지털',
 '센싱',
 '모드',
 '원적외선',
 '플리즈마',
 '발생',
 '음이온',
 '케어',
 '성분',
 '이온',
 '전자파',
 '물질',
 '방지',
 '이중안전',
 '제어',
 '시스템',
 '미열',
 '자체',
 '기능',
 '열',
 '차단',
 '유해',
 '장치',
 '기기',
 '안전',
 '케이스',
 '분리',
 '위생',
 '흡인구',
 '탈착필터',
 '부착형',
 '스무딩',
 '디퓨저',
 '스타일링',
 '부속품',
 '집중노즐',
 '콘센트레이터',
 '자석',
 '노즐',
 '손잡이',
 '고정식',
 '형태',
 '접이식',
 '코드',
 '연질',
 '길이',
 '코드릴',
 '무선',
 '가능',
 '배터리잔량표시',
 '충전',
 '스탠드',
 '기능',
 '시간',
 '무선',
 '벽걸이용',
 '거치대',
 '거치',
 '고리',
 '프리볼트',
 '방지',
 '부가',
 '고데기',
 '미끄럼',
 '기능',
 '마사지',
 '두피',
 '색상',
 '외관',
 '재질',
 '무게',
 '제품',
 '디자인',
 '그립감',
 '크기',
 '소비전력',
 '에너지',
 '효율',
 '표면',
 '제품',
 '잔기스',
 '내구성',
 '진동',
 '소음',
 '냄새',
 '소음진동',
 '포장',
 '패키지',
 '안정성',
 '디자인',
 '가격',
 '프로모션',
 '혜택',
 '가성비',
 '저가',
 '할인',
 '프리미엄',
 '대기업',
 '중소기업',
 '브랜드',
 '외국산',
 'AS',
 '정품',
 '무상']

### aspect

In [45]:
def aspect(df_ta,z_dic,ma_li):
    a_li = []
    for row,col in df_ta.iterrows():
        if col["reason"] != "":
            if set(col["reason"]).issubset(ma_li):
                key_candi = []
                for key,value in z_dic.items():
                    if set(col["reason"]).issubset(value):
                        key_candi.append(key)
                a_li.append(key_candi)
            else:
                a_li.append([end_product])
        else:
            a_li.append([end_product])
    df_ta["a"] = a_li
    return df_ta

In [47]:
end_product = "드라이기"
df_ta = df_drop.copy()
z_dic = zero_dic
ma_li = all_ma

df_aspect = aspect(df_ta,z_dic,ma_li)
df_aspect.loc[0:40]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u,a
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,1,[가격],"[저렴하다, 긍정, 1, 1]",[],[가격 및 혜택]
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,3,[간단하다],"[좋다, 긍정, 1, 12]",[],[드라이기]
2,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,4,[손쉽다],"[좋다, 긍정, 1, 12]",[],[드라이기]
3,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,5,[사용],"[좋다, 긍정, 1, 12]",[],[드라이기]
4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,6,[하다],"[좋다, 긍정, 1, 12]",[],[드라이기]
5,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"[좋다, 긍정, 1, 12]",[],[드라이기]
6,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[남성 ],[드라이기]
7,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,3,[하다],"[굿, 긍정, 1, 1]",[남성 ],[드라이기]
8,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,4,[아주],"[굿, 긍정, 1, 1]",[남성 ],[드라이기]
9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,6,[이다],"[굿, 긍정, 1, 1]",[남성 ],[드라이기]


# Delete stop words

In [61]:
# 제가 몇개만 추려서 파일을 만들었습니다.
stop_df = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\result\0120\df_aspect2.xlsx",sheet_name="Sheet2")
stop_df["reason"] = stop_df["reason"].map(lambda x : ast.literal_eval(x))
stop_df.head()

,Unnamed: 0,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u,a
0,4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,6,[하다],"['좋다', '긍정', 1, 12]",[],['드라이기']
1,5,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,7,[아주],"['좋다', '긍정', 1, 12]",[],['드라이기']
2,9,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,6,[이다],"['굿', '긍정', 1, 1]",['남성 '],['드라이기']
3,16,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,2,[아직도],"['저', '부정', -1, 1]",[],['드라이기']
4,18,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,1,4,[있다],"['저', '부정', -1, 1]",[],['드라이기']


In [62]:
stop_list = []
for i in range(len(stop_df)):
    stop_list.extend(stop_df.loc[i,"reason"])

stop_list

['하다',
 '아주',
 '이다',
 '아직도',
 '있다',
 '산지',
 '되어다',
 '약간',
 '조금',
 '되어다',
 '하니',
 '잇다',
 '찾다',
 '요',
 '뭐',
 '수',
 '있다',
 '하다',
 '함',
 '배송',
 '주문',
 '도착',
 '출발',
 '배송',
 '완료',
 '로켓',
 '와우']

In [63]:
# 지우기
def stop_remove(df_total,stop_list_):
    stop_index = []
    for row,col in df_total.iterrows():
        if col["reason"] != "":
            if set(col["reason"]).issubset(stop_list_):
                #print(col["reason"])
                stop_index.append(row)

    df_total = df_total.drop(stop_index,axis=0).reset_index(drop=True)
    return df_total

In [64]:
df_total = df_aspect.copy()
stop_list_ = stop_list

df_total = stop_remove(df_total,stop_list_)
df_total.loc[0:40]

,date,id,Product Name,review_num,sen_num,reason_num,reason,s,u,a
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,1,[가격],"[저렴하다, 긍정, 1, 1]",[],[가격 및 혜택]
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,3,[간단하다],"[좋다, 긍정, 1, 12]",[],[드라이기]
2,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,4,[손쉽다],"[좋다, 긍정, 1, 12]",[],[드라이기]
3,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,1,5,[사용],"[좋다, 긍정, 1, 12]",[],[드라이기]
4,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",1,2,2,[사용],"[굿, 긍정, 1, 1]",[남성 ],[드라이기]
5,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2,1,1,[가격],"[예쁘다, 긍정, 1, 6]",[],[가격 및 혜택]
6,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2,1,2,[대비],"[예쁘다, 긍정, 1, 6]",[],[드라이기]
7,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2,1,4,,"[좋다, 긍정, 1, 12]",[],[드라이기]
8,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,1,[가격],"[만족하다, 긍정, 1, 5]",[],[가격 및 혜택]
9,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",3,1,2,[대비],"[만족하다, 긍정, 1, 5]",[],[드라이기]


# 마지막 전처리

### Opinion_id 처리하기

In [65]:
def all_result(df_total,end_product,from_):
    reason_n = 1
    buffer_ = (1,1,reason_n)
    id_li = []

    for row,col in df_total.iterrows():
        if col["review_num"] == buffer_[0] and col["sen_num"] == buffer_[1]:
            rn_ = str(col["review_num"])
            sn_ = str(col["sen_num"])
            id_ = [end_product,rn_,sn_,str(reason_n)]
            id_li.append("-".join(id_))
        else:
            reason_n = 1
            rn_ = str(col["review_num"])
            if col["review_num"] == buffer_[0] and col["sen_num"] != buffer_[1]+1:
                sn_ = str(buffer_[1]+1)
            else:
                sn_ = str(col["sen_num"])
            id_ = [end_product,rn_,sn_,str(reason_n)]
            id_li.append("-".join(id_))
        buffer_ = (col["review_num"],col["sen_num"],reason_n)
        reason_n += 1
    df_total["Opinion_ID"] = id_li
    df_total["h"] = df_total["id"].map(lambda x: [from_,x ])
    df_total = df_total.rename(columns={"Product Name":"e","reason":"r","date":"t"})
    df_total = df_total.loc[:,["Opinion_ID","e","a","s","r","u","h","t"]]
    return df_total

In [66]:
end_product = "드라이기"
from_ = "쿠팡"

df_result = all_result(df_total,end_product,from_)
df_result.loc[0:20]

,Opinion_ID,e,a,s,r,u,h,t
0,드라이기-1-1-1,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[가격 및 혜택],"[저렴하다, 긍정, 1, 1]",[가격],[],"[쿠팡, 황*영 ]",2020.12.11
1,드라이기-1-1-2,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[좋다, 긍정, 1, 12]",[간단하다],[],"[쿠팡, 황*영 ]",2020.12.11
2,드라이기-1-1-3,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[좋다, 긍정, 1, 12]",[손쉽다],[],"[쿠팡, 황*영 ]",2020.12.11
3,드라이기-1-1-4,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[좋다, 긍정, 1, 12]",[사용],[],"[쿠팡, 황*영 ]",2020.12.11
4,드라이기-1-2-1,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[굿, 긍정, 1, 1]",[사용],[남성 ],"[쿠팡, 황*영 ]",2020.12.11
5,드라이기-2-1-1,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[가격 및 혜택],"[예쁘다, 긍정, 1, 6]",[가격],[],"[쿠팡, 이*정 ]",2020.12.04
6,드라이기-2-1-2,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[예쁘다, 긍정, 1, 6]",[대비],[],"[쿠팡, 이*정 ]",2020.12.04
7,드라이기-2-1-3,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[좋다, 긍정, 1, 12]",,[],"[쿠팡, 이*정 ]",2020.12.04
8,드라이기-3-1-1,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[가격 및 혜택],"[만족하다, 긍정, 1, 5]",[가격],[],"[쿠팡, 황*철 ]",2020.12.04
9,드라이기-3-1-2,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",[드라이기],"[만족하다, 긍정, 1, 5]",[대비],[],"[쿠팡, 황*철 ]",2020.12.04


# 혹시몰라 최종 결과물 df_result.xlsx 첨부합니다. 